# Create an OpenVINO™ Inference Engine
In this section, we will create an inference engine wrapper, a class that will get image data as input, analyze it, and return the analysis result. We will be using Intel OpenVINO™ runtime for analysis.

## Get Global Variables
First, read the previously stored variables. We need the name of the directory that will be used to store our ML solution files. If this directory does not exist, we will create a directory with a specified directory name.

In [1]:
import sys
sys.path.append('../common')
from env_variables import *

import os
print(isSolutionPath)
if not os.path.exists(isSolutionPath):
    os.mkdir(isSolutionPath)

inferenceserver


## Download the OpenVINO™ ML model
Next, we will download Intel's OpenVINO™ Intermediate Representation models (BIN + XML files) for object detection. We will download multiple models, each with different capabilities. For more details about the capabilities of these models, please refer to the link provided below.  
 
To add different models to this sample, please click on a model in the [ModelZoo page](https://github.com/opencv/open_model_zoo/blob/master/models/intel/index.md) to view its performance details and then update the `modelNames` list below.  

In this sample, we will be downloading the following models:

* person detectors
    * person-detection-retail-0002 
    * person-detection-retail-0013
    * pedestrian-detection-adas-0002
    * pedestrian-detection-adas-binary-0001
* person + vehicle detectors
    * pedestrian-and-vehicle-detector-adas-0001
    * person-vehicle-bike-detection-crossroad-0078
    * person-vehicle-bike-detection-crossroad-1016
* vehicle detectors
    * vehicle-detection-adas-0002
    * vehicle-detection-adas-binary-0001
    * vehicle-license-plate-detection-barrier-010

The following code cell will download these models from ModelZoo and store them in our ML solution directory.

> <span>[!NOTE]</span>
> The model download URLs frequently change, so if any of the code below fails, please update the source URLs accordingly.

In [2]:
# Download the model files
import os
import yaml
import urllib.request


def getModel(modelName, isSolutionPath):
    isSolutionModelFilePath = os.path.join(".", isSolutionPath, "models", modelName)

    if not os.path.exists(isSolutionModelFilePath):
        url = "https://raw.githubusercontent.com/opencv/open_model_zoo/master/models/intel/{0}/model.yml".format(modelName)
        resp = urllib.request.urlopen(url)
        yamlFileContent = resp.read()

        yamlContent = yaml.load_all(yamlFileContent)
        for c in yamlContent:
            for k1, v1 in c.items():
                if k1 == "files":
                    for k2, v2 in enumerate(v1):
                        for k in v2:
                            if k == "name":
                                modelPathName = v2[k] 
                            if k == "source":
                                modelSourceURL = v2[k]
                                # Download the model
                                pathDownload = os.path.join(isSolutionModelFilePath, modelPathName)
                                headTail = os.path.split(pathDownload) 
                                os.makedirs(headTail[0], exist_ok=True)
                                res = urllib.request.urlretrieve(modelSourceURL, pathDownload)
                                print("{} downloaded".format(pathDownload))
                                break
                    break
    else:
        print("{} already exists here, so not downloading again.".format(isSolutionModelFilePath))


# see full list of models at Model Zoo...
modelNames = [  # person detectors
                "person-detection-retail-0002", 
                "person-detection-retail-0013", 
                "pedestrian-detection-adas-0002", 
                "pedestrian-detection-adas-binary-0001", 
                # person + vehicle detectors
                "pedestrian-and-vehicle-detector-adas-0001",
                "person-vehicle-bike-detection-crossroad-0078",
                "person-vehicle-bike-detection-crossroad-1016",
                # vehicle detectors
                "vehicle-detection-adas-0002",
                "vehicle-detection-adas-binary-0001",
                "vehicle-license-plate-detection-barrier-0106",
                # license plate recognition
                "text-recognition-0012",
                "license-plate-recognition-barrier-0001"]

for modelName in modelNames:
    print("Downloading: {0}".format(modelName))
    getModel(modelName, isSolutionPath)

Downloading: person-detection-retail-0002
./inferenceserver/models/person-detection-retail-0002 already exists here, so not downloading again.
Downloading: person-detection-retail-0013
./inferenceserver/models/person-detection-retail-0013 already exists here, so not downloading again.
Downloading: pedestrian-detection-adas-0002
./inferenceserver/models/pedestrian-detection-adas-0002 already exists here, so not downloading again.
Downloading: pedestrian-detection-adas-binary-0001
./inferenceserver/models/pedestrian-detection-adas-binary-0001 already exists here, so not downloading again.
Downloading: pedestrian-and-vehicle-detector-adas-0001
./inferenceserver/models/pedestrian-and-vehicle-detector-adas-0001 already exists here, so not downloading again.
Downloading: person-vehicle-bike-detection-crossroad-0078
./inferenceserver/models/person-vehicle-bike-detection-crossroad-0078 already exists here, so not downloading again.
Downloading: person-vehicle-bike-detection-crossroad-1016
./in

If the cell above returns an issue related to the `import yaml` command, you can download the package by running `pip3 install pyyaml` based on your version of Pip. 

## Create Inference Engine Wrapper
Next, we will create a class that with different properties and methods to help scoring and analysing data from an image. This class will also help us specify analytics compute targets, such as CPU, VPU, FPGA, and debugging features.

The AnalyticsAPI class below has a parameter named `targetDev`, which can be used to specify what type of Intel® hardware acceleration should be used. You can set this parameter in the next section, when you create the container image. The four options are "CPU" (CPU acceleration), "MYRIAD" (VPU acceleration), "GPU" (GPU acceleration), and "FPGA" (FPGA acceleration). Intel® also has different plugins and extensions for your choice of hardware acceleration.

Read the following documentation to more about [Intel® accelerated hardware](https://software.intel.com/content/www/us/en/develop/tools/openvino-toolkit/hardware.html), [plugins for accelerated hardware](https://docs.openvinotoolkit.org/2018_R5/_docs_IE_DG_supported_plugins_Supported_Devices.html), and [developing with the OpenVINO™ toolkit](https://docs.openvinotoolkit.org/latest/_docs_IE_DG_Deep_Learning_Inference_Engine_DevGuide.html).

In [3]:
%%writefile $isSolutionPath/score.py

import sys
import linecache
import threading
from collections import OrderedDict
from openvino.inference_engine import IENetwork, IEPlugin
import timeit as t
import numpy as np
import logging
import json
import cv2
import os

logging.basicConfig(level=logging.DEBUG)

def PrintGetExceptionDetails():
    exType, exValue, exTraceback = sys.exc_info()

    tbFrame = exTraceback.tb_frame
    lineNo = exTraceback.tb_lineno
    fileName = tbFrame.f_code.co_filename

    linecache.checkcache(fileName)
    line = linecache.getline(fileName, lineNo, tbFrame.f_globals)

    exMessage = '[IS] Exception:\n\tFile name: {0}\n\tLine number: {1}\n\tLine: {2}\n\tValue: {3}'.format(
        fileName, lineNo, line.strip(), exValue)

    logging.info(exMessage)


class MLModel:

    # targetDev can take on the values of "CPU", "MYRIAD", "GPU", "FPGA"
    def __init__(self, modelName, modelPrecision, targetDev, pluginPath=None, cpuExtensions=None):
        try:
            self.initialized = False
            self.modelPath = "./models"
            self.modelName = modelName
            self.modelPrecision = modelPrecision
            self.targetDev = targetDev
            self.pluginPath = pluginPath
            self.cpuExtensions = cpuExtensions
            self._lock = threading.Lock()
            self.initEngine()

        except:
            PrintGetExceptionDetails()

    def initEngine(self):
        try:
            with self._lock:
                self.modelXMLFileName = os.path.join(
                    self.modelPath, self.modelName, self.modelPrecision, self.modelName + ".xml")
                self.modelBINFileName = os.path.join(
                    self.modelPath, self.modelName, self.modelPrecision, self.modelName + ".bin")

                # Initiate targetDev for hardware acceleration. Default is CPU acceleration
                self.iePlugin = IEPlugin(
                    device=self.targetDev, plugin_dirs=self.pluginPath)
                if self.cpuExtensions and 'CPU' in self.targetDev:
                    self.iePlugin.add_cpu_extension(self.cpuExtensions)

                ieNet = IENetwork(model=self.modelXMLFileName,
                                  weights=self.modelBINFileName)

                assert len(ieNet.inputs.keys()
                           ) == 1, "Only single input topologies supported!"
                assert len(
                    ieNet.outputs) == 1, "Only single output topologies supported!"
                self.inputBlob = next(iter(ieNet.inputs))
                self.outBlob = next(iter(ieNet.outputs))
                self.ieExecNet = self.iePlugin.load(
                    network=ieNet, num_requests=2)

                # Read and pre-process input image
                # n, c, h, w
                self.ieNetShape = ieNet.inputs[self.inputBlob].shape

        except:
            PrintGetExceptionDetails()

    def preprocess(self, cvImage):
        try:
            ih, iw = cvImage.shape[:-1]
            imageHW = (ih, iw)

            if (ih, iw) != (self.ieNetShape[2], self.ieNetShape[3]):
                cvImage = cv2.resize(
                    cvImage, (self.ieNetShape[3], self.ieNetShape[2]))

            # Change data layout from HWC to CHW
            cvImage = cvImage.transpose((2, 0, 1))
            cvImage = cvImage.reshape(self.ieNetShape)

            return cvImage
        except:
            PrintGetExceptionDetails()

    def postprocess(self, infRes):
        try:
            detectedObjects = []

            for obj in infRes[self.outBlob][0][0]:
               dobj = {
                   "type": "entity",
                   "entity": {
                       "tag": {
                           "value": str(obj[1]),
                           "confidence": str(obj[2])
                       },
                       "box": {
                           "l": str(obj[3]),
                           "t": str(obj[4]),
                           "w": str(obj[5]-obj[3]),
                           "h": str(obj[6]-obj[4])
                       }
                   }
               }
               detectedObjects.append(dobj)

            return detectedObjects

        except:
            PrintGetExceptionDetails()

    def score(self, cvImage):
        try:
            with self._lock:
                image = self.preprocess(cvImage)
                infRes = self.ieExecNet.infer(inputs={self.inputBlob: image})

            return self.postprocess(infRes)

        except:
            PrintGetExceptionDetails()

    def about(self):
        aboutString = "ModelName: {0}<br>ModelPrecision: {1}<br>TargetDev: {2}<br>PluginPath: {3}<br>CpuExtensions: {4}".format(
            self.modelName, self.modelPrecision, self.targetDev, self.pluginPath, self.cpuExtensions)
        return aboutString


Overwriting inferenceserver/score.py


The `score` method of the inference engine class above will return a dictionary of inferences in the following form:

```
      {
        "type": "entity",
        "entity": {
          "tag": {
            "value": "1.0",
            "confidence": "0.9596136"
          },
          "box": {
            "l": "0.69242793",
            "t": "0.3647236",
            "w": "0.08401036",
            "h": "0.07765585"
          }
        }
      },
      {
        "type": "entity",
        "entity": {
          "tag": {
            "value": "1.0",
            "confidence": "0.92975134"
          },
          "box": {
            "l": "0.5211431",
            "t": "0.44633362",
            "w": "0.16630614",
            "h": "0.12689856"
          }
        }
      }
```

## Next Steps
If all the code cells above have successfully finished running, return to the Readme page to continue.   
